## 데이터 준비와 전처리


In [39]:
import pandas as pd

In [40]:
import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [41]:
ratings.shape

(1000209, 4)

In [42]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [43]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [44]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [45]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [46]:
# 데이터셋 합치기
ratings = ratings.join(movies.set_index('movie_id'), on='movie_id')
ratings = ratings.drop(columns=['timestamp', 'genre'])
ratings.head()

,user_id,movie_id,counts,title
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975)
1,1,661,3,James and the Giant Peach (1996)
2,1,914,3,My Fair Lady (1964)
3,1,3408,4,Erin Brockovich (2000)
4,1,2355,5,"Bug's Life, A (1998)"


#### 유저 수

In [47]:
ratings['user_id'].nunique()

6039

#### 영화 수

In [48]:
ratings['movie_id'].nunique()

3628

#### 인기 많은 영화

In [49]:
movie_count = ratings.groupby('title')['counts'].count()

In [50]:
movie_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

#### 가장 인기 많은 영화는 코미디 장르의 American Beauty (1999)이며, 스타워즈가 상위 5개중 3개를 차지하고 있다. 

In [51]:
# 유저별 몇 개의 영화를 보고있는지에 대한 통계
user_count =ratings.groupby('user_id')['movie_id'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: movie_id, dtype: float64

In [52]:
# 유저별 시청횟수 중앙값에 대한 통계
user_median = ratings.groupby('user_id')['counts'].median()
user_median.describe()

count    6039.000000
mean        4.055970
std         0.432143
min         3.000000
25%         4.000000
50%         4.000000
75%         4.000000
max         5.000000
Name: counts, dtype: float64

### 선호하는 영화 5가지 추가

In [53]:
# 선호하는 영화 5편 
my_favorite_title = ['Toy Story (1995)' , 'E.T. the Extra-Terrestrial (1982)' ,'Back to the Future (1985) ' ,
               'Men in Black (1997)','Terminator 2: Judgment Day (1991)']
my_favorite_id = [2000, 2102, 2460, 2297, 2509 ]

# 'kyungmo'이라는 user_id가 위 아티스트의 노래를 30회씩 들었다고 가정하겠습니다.
my_playlist = pd.DataFrame({'user_id': ['kyungmo']*5,'movie_id': my_favorite_id, 'title': my_favorite_title, 'counts':[30]*5})

if not ratings.isin({'user_id':['kyungmo']})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
     ratings = ratings.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

ratings.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,movie_id,counts,title
1000203,6040,1090,3,Platoon (1986)
1000205,6040,1094,5,"Crying Game, The (1992)"
1000206,6040,562,5,Welcome to the Dollhouse (1995)
1000207,6040,1096,4,Sophie's Choice (1982)
1000208,6040,1097,4,E.T. the Extra-Terrestrial (1982)
0,kyungmo,2000,30,Toy Story (1995)
1,kyungmo,2102,30,E.T. the Extra-Terrestrial (1982)
2,kyungmo,2460,30,Back to the Future (1985)
3,kyungmo,2297,30,Men in Black (1997)
4,kyungmo,2509,30,Terminator 2: Judgment Day (1991)


### 모델에 활용하기 위한 전처리

In [54]:
from scipy.sparse import csr_matrix
#유저 , 영화 고유한 값 개수
user_unique = ratings['user_id'].unique()
movie_unique = ratings['title'].unique()

# 고유한 값의 개수만큼 유저 id와 영화 제목 인덱싱
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [57]:
# 인덱싱이 잘 되었는지 확인 
print(user_to_idx['kyungmo'])    
print(movie_to_idx['Men in Black (1997)'])

6039
175


In [58]:
# 데이터 컬럼 내 값을 indexing된 값으로 교체

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# movie_to_idx을 통해 title 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = ratings['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print(' column indexing OK!!')
    ratings['title'] = temp_movie_data
else:
    print('column indexing Fail!!')

ratings

user_id column indexing OK!!
 column indexing OK!!


,user_id,movie_id,counts,title
0,0,1193,5,0
1,0,661,3,1
2,0,914,3,2
3,0,3408,4,3
4,0,2355,5,4
...,...,...,...,...
0,6039,2000,30,40
1,6039,2102,30,26
2,6039,2460,30,3628
3,6039,2297,30,175


## CSR matrix

In [61]:
from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
num_movie = ratings['title'].nunique()

csr_data = csr_matrix((ratings.counts, (ratings.user_id, ratings.title)), shape= (num_user, num_movie))
csr_data

<6040x3629 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

## AlternatingLeastSquares 모델 훈련

In [62]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [63]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [64]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3629x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [65]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

In [66]:
kyungmo, men_in_black = user_to_idx['kyungmo'], movie_to_idx['Men in Black (1997)']
kyungmo_vector, men_vector = als_model.user_factors[kyungmo], als_model.item_factors[men_in_black]


In [67]:
kyungmo_vector

array([-0.19663584, -0.42425483, -0.04455297,  0.55843616,  0.40189868,
       -1.1693351 , -0.14045641,  0.52637535,  1.1660562 ,  0.33130884,
       -0.0984352 ,  1.0975572 ,  1.735413  , -0.84812295,  0.17253418,
        1.5810263 ,  0.6838569 , -1.3207712 , -0.53366345, -0.48389247,
       -1.1901453 , -0.13437009,  0.50243914, -0.4965004 , -0.29817548,
        0.8844903 , -2.2587662 ,  0.06345198, -0.11574789,  0.05195914,
        0.11340105,  0.09611266,  0.36721453,  0.47608274, -0.01080915,
       -2.1571827 , -0.44652852, -0.49070436,  0.7260644 , -0.09933724,
        1.6354102 ,  0.7844046 ,  1.3468038 , -0.49654555, -0.38173705,
       -0.36083767,  0.00318237,  0.3744083 ,  0.39231625, -1.4855132 ,
        1.1961434 ,  0.8298734 , -1.1583258 , -0.31346726, -1.2616665 ,
        0.35950917, -1.651881  , -0.33099562,  1.5323496 ,  0.06002386,
        0.11386656, -0.21333271,  0.5285615 ,  0.43214226,  0.03448948,
       -0.3305134 , -0.15739632,  1.679505  ,  0.05809692,  0.60

In [68]:
men_vector

array([ 0.00970846, -0.02553955, -0.00324733,  0.01140641,  0.01234274,
       -0.00753134,  0.00726507,  0.01519088,  0.01321342, -0.0125287 ,
       -0.02093395,  0.02748109,  0.03129083, -0.00392464,  0.03953335,
        0.03369604,  0.0408472 , -0.00714218,  0.01622903, -0.00343379,
       -0.00298336,  0.01030478, -0.00365068,  0.00122145, -0.01181894,
       -0.00162506, -0.02285807, -0.00273972,  0.01330745,  0.04102079,
       -0.00746022,  0.00391916, -0.01274214,  0.00967189, -0.00681683,
       -0.014449  , -0.00544339,  0.01168188, -0.01732298,  0.00099242,
        0.03494279,  0.01800469,  0.03681023,  0.0031385 , -0.01077711,
        0.00035756, -0.00656186,  0.01785514,  0.01909992, -0.02413089,
        0.00635303,  0.03610375, -0.01722034,  0.00959817, -0.02093729,
        0.05760978,  0.01101706, -0.00603743,  0.04352606,  0.00475627,
        0.0075998 ,  0.012164  ,  0.00910342,  0.01007882, -0.00932601,
        0.0086917 ,  0.03794054,  0.04567502,  0.00246011, -0.00

In [69]:
np.dot(kyungmo_vector, men_vector)

0.9222874

#### 1에 가까운 0.92가 내적 결과로 출력되었다. 우리 모델이 잘 학습되었음을 알 수 있다.

#### 그렇다면, 내가 선호하는 영화의 장르와 동떨어진 영화의 선호도를 측정해보자

In [71]:
love = movie_to_idx['Shakespeare in Love (1998)']
love_vector = als_model.item_factors[love]
np.dot(kyungmo_vector, love_vector)

0.28566307

#### 선호도가 낮게 나오는것을 확인할 수 있었다.

## 내가 좋아하는 영화와 비슷한 영화 찾기

In [74]:
favorite_movie = 'Men in Black (1997)'
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(175, 0.9999999),
 (107, 0.84177077),
 (92, 0.62135476),
 (62, 0.59969705),
 (150, 0.49115545),
 (145, 0.49071336),
 (82, 0.48338518),
 (124, 0.4725806),
 (542, 0.45151624),
 (375, 0.44254008),
 (3628, 0.43077862),
 (3466, 0.41285792),
 (138, 0.39359576),
 (685, 0.37099516),
 (1045, 0.36551642)]

In [76]:
#movie_to_idx 를 뒤집어, index로부터 영화 이름을 얻는 dict를 생성
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

['Men in Black (1997)',
 'Jurassic Park (1993)',
 'Terminator 2: Judgment Day (1991)',
 'Total Recall (1990)',
 'Independence Day (ID4) (1996)',
 'Fifth Element, The (1997)',
 'Lost World: Jurassic Park, The (1997)',
 'Matrix, The (1999)',
 'Rocky Horror Picture Show, The (1975)',
 'Face/Off (1997)',
 'Back to the Future (1985) ',
 'Schlafes Bruder (Brother of Sleep) (1995)',
 'True Lies (1994)',
 'Planet of the Apes (1968)',
 'Grumpy Old Men (1993)']

#### 모든 영화를 다 알지는 못하지만, 얼추 비슷한 장르의 영화가 추천된 것 같다. 

In [77]:
# 함수로 만들기
def get_similar_movie(movie_name: str):
    movie_id = movie_to_idx[movie_name]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie



#### 로맨스 영화로 설정해보자

In [80]:
get_similar_movie('Shakespeare in Love (1998)')

['Shakespeare in Love (1998)',
 'Groundhog Day (1993)',
 'Election (1999)',
 'American Beauty (1999)',
 'Being John Malkovich (1999)',
 'Elizabeth (1998)',
 'Four Weddings and a Funeral (1994)',
 'For Ever Mozart (1996)',
 'Ghostbusters (1984)',
 'Toy Story 2 (1999)']

## 내가 좋아할만한 영화 추천받기

In [82]:
user = user_to_idx['kyungmo']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(107, 0.8736303),
 (124, 0.7522327),
 (50, 0.63712966),
 (62, 0.55844593),
 (33, 0.5217565),
 (4, 0.47783124),
 (322, 0.4495123),
 (22, 0.4259818),
 (160, 0.39147544),
 (141, 0.37650928),
 (10, 0.37547332),
 (110, 0.3626419),
 (317, 0.35897198),
 (330, 0.3472137),
 (200, 0.331775),
 (284, 0.32314152),
 (479, 0.3215531),
 (145, 0.30241975),
 (0, 0.30195257),
 (685, 0.29642463)]

In [83]:
[idx_to_movie[i[0]] for i in movie_recommended]

['Jurassic Park (1993)',
 'Matrix, The (1999)',
 'Toy Story 2 (1999)',
 'Total Recall (1990)',
 'Aladdin (1992)',
 "Bug's Life, A (1998)",
 'Babe (1995)',
 'Back to the Future (1985)',
 'Forrest Gump (1994)',
 'Fugitive, The (1993)',
 'Beauty and the Beast (1991)',
 'Groundhog Day (1993)',
 'Twelve Monkeys (1995)',
 'Lion King, The (1994)',
 'Terminator, The (1984)',
 'Nightmare Before Christmas, The (1993)',
 'Contact (1997)',
 'Fifth Element, The (1997)',
 "One Flew Over the Cuckoo's Nest (1975)",
 'Planet of the Apes (1968)']

#### 내가 설정한 영화들과 비슷한 장르의 영화들이 추천되는것을 확인할 수 있다.

### 기여 정도 확인

In [84]:
jurassic = movie_to_idx['Jurassic Park (1993)']
explain = als_model.explain(user, csr_data, itemid=jurassic)

In [85]:
[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[('Men in Black (1997)', 0.5730779004279495),
 ('Terminator 2: Judgment Day (1991)', 0.28642412466198214),
 ('Back to the Future (1985) ', 0.023268825523740624),
 ('E.T. the Extra-Terrestrial (1982)', 0.017895399834067353),
 ('Toy Story (1995)', -0.047704666197631834)]

#### 맨인블랙과 터미네이터가 가장 큰 기여를 하였다. 토이스토리의 경우 음수가 나온걸로보아 기여를 하지 못한것 같다.

## 회고


작년 학교에서 수강한 기계학습 수업에서 SVD라는 행렬분해 방법을 배운적이 있다. 그때 교수님께서 이 노드와 같은 '영화 추천'에 대한 내용으로 설명을 해주셨는데, 그때는 영화 행렬과 유저 행렬의 새로운 feature를 만든다는것에 대해 이해를 잘 하지 못하고 그저 수학적인 내용으로만 이해를 했었다.
하지만 이번 아이펠 노드를 통해 행렬분해가 무엇이고, 어떤 원리를 통해 진행되는지 알 수 있었다. 

이번노드를 통해 우리가 흔히 말하는 '알고리즘'이 대략적으로 어떤 방식으로 돌아가는지 조금이나마 배울 수 있었고, 그래프 기반 추천 시스템에도 관심이 생겼다.

